## Clustering: Explanation
The objective is to create a structured dataset to train the model. The process goes through three phases of hierarchical clustering that build an increasingly complex representation: first, individual people are isolated and categorized, then these poses are combined to represent the overall situation of the image, and finally the temporal dimension and sensory context are added to capture behavioral routines. This progression from particular to general allows building a reusable vocabulary at each level, reducing computational complexity and improving generalization of the final models.

For person detection, two versions of YOLO are used: a standard one and one specialized for grayscale (for night mode). The grayscale model can be trained following the guide available at https://docs.ultralytics.com/it/datasets/detect/coco8-grayscale/#usage. CLIP extracts semantic features from detected people, while K-means groups similar elements (very often centroids are also leveraged to order clusters by similarity).

The result is a tripartite dataset: vocabulary of base poses, library of activities as meaningful combinations of poses, and contextualized routines with sensory correlations. This hierarchical structure enables training models that understand human behavior at multiple temporal scales, from instantaneous poses to complete daily routines.

Use "reviewer.js" to revise clusters

In [ ]:
!pip install numpy==2.2.6 pandas==2.3.0 scikit-learn==1.7.0 pillow torch==2.3.0 torchvision==0.18.0 open-clip-torch==2.32.0 ultralytics==8.3.165 tqdm opencv-python==4.11.0.86

# PYTHON VERSION USED: 3.10.17

In [ ]:
import os
from datetime import datetime, timedelta
import shutil 
import numpy as np
from PIL import Image, ImageEnhance
import torch
from tqdm.auto import tqdm
import pickle
import cv2
from collections import defaultdict, deque
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import gc
import time

import open_clip
from ultralytics import YOLO
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
# import timm -> for ViT (not used) -> pip install timm


from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

from scipy.spatial.distance import cdist
from scipy.cluster.hierarchy import linkage, leaves_list


device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

### Definizione di costanti, modello e funzioni di utilità

In [ ]:
BASE_DIR = "Home Assistant Monitor Complete Jul 12 2025"
PATH_IMAGES = os.path.join(BASE_DIR, "images")
PATH_CSV = os.path.join(BASE_DIR, "stati_home_assistant.csv")

yolo_model = YOLO("yolo12l.pt") 
yolo_model = yolo_model.to(device).eval()

yolo_model_grayscale = YOLO('yolo12l-grayscale.pt')
yolo_model_grayscale = yolo_model_grayscale.to(device).eval() # to load on MPS install torch==2.3.0 torchvision==0.18.0 (just indicated in requirements)

clip_model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
clip_model = clip_model.to(device).eval()

# ViT - not used in this version
# vit = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=0)
# vit = vit.to(device).eval()

In [ ]:
def expand_bbox(bbox, img_w, img_h, expansion_ratio=0.3):
    xmin, ymin, xmax, ymax = bbox
    w, h = xmax - xmin, ymax - ymin
    pad_w, pad_h = w * expansion_ratio, h * expansion_ratio
    xmin_e = max(int(xmin - pad_w), 0)
    ymin_e = max(int(ymin - pad_h), 0)
    xmax_e = min(int(xmax + pad_w), img_w - 1)
    ymax_e = min(int(ymax + pad_h), img_h - 1)
    return xmin_e, ymin_e, xmax_e, ymax_e


def detect_grayscale(image):
    if image.mode == 'L':
        return True
    elif image.mode == 'RGB':
        img_array = np.array(image)
        r, g, b = img_array[:,:,0], img_array[:,:,1], img_array[:,:,2]
        return np.array_equal(r, g) and np.array_equal(g, b)
    return False

def enhance_grayscale(image):
    if image.mode != 'L':
        image = image.convert('L')
    
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.4)
    
    img_array = np.array(image)
    img_array = cv2.equalizeHist(img_array)
    image = Image.fromarray(img_array)
    
    return Image.merge('RGB', (image, image, image))

def extract_clip_embedding(image, is_grayscale=False):
    if is_grayscale:
        embeddings = []
        
        img_tensor = preprocess(image).unsqueeze(0).to(device)
        with torch.no_grad():
            feat = clip_model.encode_image(img_tensor)
            feat = feat / feat.norm(dim=-1, keepdim=True)
        embeddings.append(feat.squeeze().cpu().numpy())
        
        enhancer = ImageEnhance.Contrast(image)
        img_enhanced = enhancer.enhance(1.2)
        img_tensor = preprocess(img_enhanced).unsqueeze(0).to(device)
        with torch.no_grad():
            feat = clip_model.encode_image(img_tensor)
            feat = feat / feat.norm(dim=-1, keepdim=True)
        embeddings.append(feat.squeeze().cpu().numpy())
        
        final_embedding = np.mean(embeddings, axis=0)
        final_embedding = final_embedding / np.linalg.norm(final_embedding)
        return final_embedding
    else:
        img_tensor = preprocess(image).unsqueeze(0).to(device)
        with torch.no_grad():
            feat = clip_model.encode_image(img_tensor)
            feat = feat / feat.norm(dim=-1, keepdim=True)
        return feat.squeeze().cpu().numpy()


def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = area1 + area2 - inter_area
    return inter_area / union_area if union_area > 0 else 0.0

def group_overlapping_bboxes(bboxes, threshold=0.5):
    n = len(bboxes)
    graph = defaultdict(list)
    for i in range(n):
        for j in range(i + 1, n):
            if calculate_iou(bboxes[i][:4], bboxes[j][:4]) >= threshold:
                graph[i].append(j)
                graph[j].append(i)
    visited = [False] * n
    groups = []
    for i in range(n):
        if not visited[i]:
            queue = deque([i])
            group = []
            while queue:
                node = queue.popleft()
                if visited[node]:
                    continue
                visited[node] = True
                group.append(bboxes[node])
                queue.extend(graph[node])
            if len(group) > 1:
                groups.append(group)
    return groups

def merge_overlapping_boxes(bboxes, threshold=0.5):
    overlapping_groups = group_overlapping_bboxes(bboxes, threshold)
    merged = []

    used = set()
    for group in overlapping_groups:
        x1 = min(b[0] for b in group)
        y1 = min(b[1] for b in group)
        x2 = max(b[2] for b in group)
        y2 = max(b[3] for b in group)
        score = np.mean([b[4] for b in group])
        merged.append([x1, y1, x2, y2, score])
        used.update(tuple(b) for b in group)

    for b in bboxes:
        if tuple(b) not in used:
            merged.append(b)

    return merged



"""

def extract_scene_embedding(image_path):
    pil_img = Image.open(image_path).convert("RGB")
    pil_img = pil_img.resize((224, 224), Image.LANCZOS)
    img_tensor = torch.from_numpy(np.array(pil_img)).permute(2, 0, 1).float() / 255.0
    img_tensor = img_tensor.unsqueeze(0).to(device)

    
    with torch.no_grad():
        features = vit(img_tensor)  # 768 dimensioni
    
    return features.squeeze().cpu().numpy()
    
"""

### STEP 1: Image Processing and Activity Clustering
Instead of treating each person as a unique entity, reusable activity categories (poses) are created. This drastically reduces complexity: from thousands of different people to a few dozen activity-types (sitting at desk with computer, standing while tidying the room, lying in bed with phone).
Manual review with revisor.js serves to clean up the automated results. The clusters_kmeans_1000 folder is configured by disabling group mode and annotations. The objective is to obtain semantically coherent poses that will become the building blocks of subsequent steps.

#### Image processing and reading

In [ ]:
image_info = []
for root, _, files in os.walk(PATH_IMAGES):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            try:
                ts = os.path.splitext(file)[0]
                dt = datetime.strptime(ts, "%d-%m_%H-%M-%S")
                image_info.append((dt, os.path.join(root, file)))
            except ValueError:
                continue
image_info.sort()

all_image_paths = [p for _, p in image_info]

print(len(all_image_paths))

In [ ]:
def prepare_quadrants_metadata(w, h, overlap_ratio=0.10):
    overlap_x = int(w * overlap_ratio)
    overlap_y = int(h * overlap_ratio)
    half_w = w // 2
    half_h = h // 2

    quadrants = [
        ("q1", (0, 0, half_w + overlap_x, half_h + overlap_y), (0, 0)),
        ("q2", (half_w - overlap_x, 0, w, half_h + overlap_y), (half_w - overlap_x, 0)),
        ("q3", (0, half_h - overlap_y, half_w + overlap_x, h), (0, half_h - overlap_y)),
        ("q4", (half_w - overlap_x, half_h - overlap_y, w, h), (half_w - overlap_x, half_h - overlap_y)),
    ]
    
    corrected_quadrants = []
    for quad_name, crop_coords, offset in quadrants:
        crop_coords = (
            max(0, crop_coords[0]),
            max(0, crop_coords[1]), 
            min(w, crop_coords[2]),
            min(h, crop_coords[3])
        )
        corrected_quadrants.append((quad_name, crop_coords, offset))
    
    return corrected_quadrants

def prepare_quadrants(pil_img, overlap_ratio=0.10):
    w, h = pil_img.size
    return prepare_quadrants_metadata(w, h, overlap_ratio)

def process_quadrants_batch(metadata_list, model, device, is_grayscale_batch=False):
    all_results = []
    
    conf_threshold = 0.4 if is_grayscale_batch else 0.5
    
    quad_images = []
    quad_metadata = []
    
    for img_metadata in metadata_list:
        img_path = img_metadata['path']
        quadrants = img_metadata['quadrants']
        
        pil_img = Image.open(img_path).convert("RGB")
        
        for _, crop_coords, offset in quadrants:
            quad_img = pil_img.crop(crop_coords)
            quad_images.append(quad_img)
            quad_metadata.append((img_path, offset, conf_threshold))
    
    if quad_images:
        batch_size = 16
        
        batch_results = []
        for i in range(0, len(quad_images), batch_size):
            batch_imgs = quad_images[i:i+batch_size]
            with torch.no_grad():
                results = model(batch_imgs, verbose=False)
                batch_results.extend(results)
        
        for result, (img_path, offset, conf_threshold) in zip(batch_results, quad_metadata):
            img_detections = []
            for bbox, cls, conf in zip(result.boxes.xyxy.cpu().numpy(), 
                                     result.boxes.cls.cpu().numpy(), 
                                     result.boxes.conf.cpu().numpy()):
                if int(cls) == 0 and conf >= conf_threshold:
                    xmin, ymin, xmax, ymax = bbox
                    xmin += offset[0]
                    xmax += offset[0]
                    ymin += offset[1]
                    ymax += offset[1]
                    img_detections.append([xmin, ymin, xmax, ymax, conf])
            
            all_results.append((img_path, img_detections))
    
    return all_results

def extract_embeddings_parallel(detection_data, max_workers=4, expansion=0.4):    
    def process_single_detection(args):
        img_path, pil_img, bbox, conf, is_grayscale = args
        xmin, ymin, xmax, ymax = bbox
        
        if xmax <= xmin or ymax <= ymin:
            return None
            
        crop = pil_img.crop((
            max(0, xmin - expansion * (xmax - xmin)),
            max(0, ymin - expansion * (ymax - ymin)),
            min(pil_img.width, xmax + expansion * (xmax - xmin)),
            min(pil_img.height, ymax + expansion * (ymax - ymin))
        ))

        if crop.width == 0 or crop.height == 0:
            return None

        emb = extract_clip_embedding(crop, is_grayscale)
        
        return {
            "bbox": [xmin, ymin, xmax, ymax],
            "embedding": emb,
            "is_grayscale": is_grayscale,
            "confidence": conf
        }
    
    results = {}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_path = {}
        
        for img_path, detection_list in detection_data.items():
            for detection_args in detection_list:
                future = executor.submit(process_single_detection, detection_args)
                if img_path not in future_to_path:
                    future_to_path[img_path] = []
                future_to_path[img_path].append(future)
        
        for img_path, futures in future_to_path.items():
            results[img_path] = []
            for future in concurrent.futures.as_completed(futures):
                result = future.result()
                if result is not None:
                    results[img_path].append(result)
    
    return results

def process_images_optimized(batch_size_normal=8, batch_size_grayscale=16, checkpoint_every=1000, checkpoint_file="checkpoint.pkl", expansion=0.4):

    if os.path.exists(checkpoint_file):
        print(f"Caricamento checkpoint da {checkpoint_file}...")
        with open(checkpoint_file, 'rb') as f:
            checkpoint_data = pickle.load(f)
        processed_paths = set(checkpoint_data.get('processed_paths', []))
        final_results = checkpoint_data.get('results', {})
    else:
        processed_paths = set()
        final_results = {}
    
    remaining_paths = [path for path in all_image_paths if path not in processed_paths]
    
    if not remaining_paths:
        return final_results
    
    chunk_size = 5000  
    
    for chunk_start in range(0, len(remaining_paths), chunk_size):
        chunk_end = min(chunk_start + chunk_size, len(remaining_paths))
        chunk_paths = remaining_paths[chunk_start:chunk_end]
        
        print(f"Processando chunk {chunk_start//chunk_size + 1}/{(len(remaining_paths)-1)//chunk_size + 1}")
        
        chunk_grayscale = []
        chunk_normal = []
        
        for img_path in tqdm(chunk_paths, desc=f"Preprocessing chunk {chunk_start//chunk_size + 1}"):
            try:
                with Image.open(img_path) as pil_img:
                    pil_img = pil_img.convert("RGB")
                    is_grayscale = detect_grayscale(pil_img)
                    
                    w, h = pil_img.size
                    quadrant_metadata = prepare_quadrants_metadata(w, h)
                
                img_metadata = {
                    'path': img_path,
                    'size': (w, h),
                    'is_grayscale': is_grayscale,
                    'quadrants': quadrant_metadata
                }
                
                if is_grayscale:
                    chunk_grayscale.append(img_metadata)
                else:
                    chunk_normal.append(img_metadata)
                    
            except Exception as e:
                print(f"Errore processando {img_path}: {e}")
                continue
        
        chunk_results = process_chunk(chunk_normal, chunk_grayscale, yolo_model, yolo_model_grayscale,
                                    batch_size_normal, batch_size_grayscale, expansion)
        
        final_results.update(chunk_results)
        
        processed_paths.update([metadata['path'] for metadata in chunk_normal + chunk_grayscale])
        
        if len(processed_paths) % checkpoint_every < chunk_size:
            save_checkpoint(checkpoint_file, processed_paths, final_results)
            print(f"Checkpoint salvato: {len(processed_paths)} immagini processate")
        
        del chunk_normal, chunk_grayscale, chunk_results
        gc.collect()
        if torch.backends.mps.is_available():
            torch.mps.empty_cache()
    
    save_checkpoint(checkpoint_file, processed_paths, final_results)
    
    return final_results

def save_checkpoint(checkpoint_file, processed_paths, results):
    checkpoint_data = {
        'processed_paths': list(processed_paths),
        'results': results,
        'timestamp': time.time()
    }
    with open(checkpoint_file, 'wb') as f:
        pickle.dump(checkpoint_data, f)

def process_chunk(normal_metadata, grayscale_metadata, yolo_model, yolo_model_grayscale, 
                 batch_size_normal, batch_size_grayscale, expansion=0.4):
    all_detections = {}
    
    if normal_metadata:
        for i in tqdm(range(0, len(normal_metadata), batch_size_normal), desc="Batch normali", leave=False):
            batch = normal_metadata[i:i+batch_size_normal]
            batch_results = process_quadrants_batch(batch, yolo_model, 'mps', False)
            
            for img_path, detections in batch_results:
                if img_path not in all_detections:
                    all_detections[img_path] = []
                all_detections[img_path].extend(detections)
            
            if torch.backends.mps.is_available():
                torch.mps.empty_cache()
    
    if grayscale_metadata:
        for i in tqdm(range(0, len(grayscale_metadata), batch_size_grayscale), desc="Batch grayscale", leave=False):
            batch = grayscale_metadata[i:i+batch_size_grayscale]
            batch_results = process_quadrants_batch(batch, yolo_model_grayscale, 'mps', True)
            
            for img_path, detections in batch_results:
                if img_path not in all_detections:
                    all_detections[img_path] = []
                all_detections[img_path].extend(detections)
    
    detection_data_for_embeddings = {}
    iou_threshold = 0.05
    
    for img_path in all_detections.keys():
        pil_img = Image.open(img_path).convert("RGB")
        is_grayscale = detect_grayscale(pil_img)
        
        merged_detections = merge_overlapping_boxes(all_detections[img_path], threshold=iou_threshold)
        
        if is_grayscale:
            pil_img = enhance_grayscale(pil_img)
        
        detection_args_list = []
        for xmin, ymin, xmax, ymax, conf in merged_detections:
            detection_args_list.append((img_path, pil_img, [xmin, ymin, xmax, ymax], conf, is_grayscale))
        
        detection_data_for_embeddings[img_path] = detection_args_list
    
    chunk_results = extract_embeddings_parallel(detection_data_for_embeddings, max_workers=4, expansion=expansion)
    
    return chunk_results


CHECKPOINT_FILE = "processing_checkpoint_70k.pkl"
BATCH_SIZE = 16
CHECKPOINT_EVERY = 500  

person_detections_per_image = process_images_optimized(
    batch_size_normal=BATCH_SIZE,
    batch_size_grayscale=BATCH_SIZE,
    checkpoint_every=CHECKPOINT_EVERY,
    checkpoint_file=CHECKPOINT_FILE
)

In [ ]:
to_save = dict()


for image_path in all_image_paths:
    if image_path not in person_detections_per_image:
        to_save[image_path] = []
        continue
    values = person_detections_per_image[image_path]
    to_save[image_path] = values.copy()
    
    for val in to_save[image_path]:
        val["crop"] = None
        val["original"] = None

    

with open("person_detections_per_image.pkl", "wb") as f:
    pickle.dump(to_save, f)

#### Cluster Division

In [ ]:
with open("person_detections_per_image.pkl", "rb") as f:
    person_detections_per_image = pickle.load(f)

In [ ]:
features = []

for i, key in tqdm(enumerate(person_detections_per_image), total=len(person_detections_per_image)):
    values = person_detections_per_image[key]
    values_sorted = sorted(
        values,
        key=lambda v: (v["bbox"][2] - v["bbox"][0]) * (v["bbox"][3] - v["bbox"][1]),
        reverse=True 
    )

    if len(values_sorted) == 0:
        continue

    for val in values_sorted:
        features.append(val["embedding"])
        val["feature index"] = len(features) - 1    
        val["path"] = all_image_paths[i]


features = np.array(features)

features.shape

In [ ]:
features_std = StandardScaler().fit_transform(features)
pca = PCA(n_components=min(features.shape[0], 20), random_state=42)
features_pca = pca.fit_transform(features_std)

In [ ]:
# To calculate the perfect number of clusters - recommended anyway to consider a high number of clusters

inertias = []
K_range = range(4, 16, 2)
silhouette_scores = []

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(features_pca)
    inertias.append(km.inertia_)

    labels_tmp = km.labels_
    n_labels = len(set(labels_tmp))

    if n_labels > 1 and n_labels < len(features_pca):
        score = silhouette_score(features_pca, labels_tmp)
    else:
        score = float('nan')
    silhouette_scores.append(score)

plt.plot(K_range, inertias, 'o-')
plt.xlabel('k')
plt.ylabel('Inertia')
plt.title('Elbow method')
plt.show()

plt.plot(K_range, silhouette_scores, 'o-')
plt.xlabel('k')
plt.ylabel('Silhouette Score')
plt.title('Silhouette analysis')
plt.show()

In [ ]:
k = 270 
kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
labels_persons = kmeans.fit_predict(features_pca)

centroids = kmeans.cluster_centers_
dists = cdist(centroids, centroids)

linkage_matrix = linkage(centroids, method='ward')
ordered_cluster_ids = leaves_list(linkage_matrix) 

label_map = {old: new for new, old in enumerate(ordered_cluster_ids)}
labels_persons = np.array([label_map[label] for label in labels_persons])


In [ ]:
from collections import defaultdict

output_dir = "clusters_kmeans_1000"

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    
os.makedirs(output_dir)

index_to_cluster = {i: label for i, label in enumerate(labels_persons)}

clusters = defaultdict(list)

for key, values in person_detections_per_image.items():
    if len(values) == 0:
        continue
    for val in values:
        cluster_id = index_to_cluster.get(val["feature index"])
        if cluster_id is not None:
            clusters[cluster_id].append((val, key))

for cluster_id, items in tqdm(clusters.items(), desc="Saving clusters"):
    cluster_folder = os.path.join(output_dir, f"cluster_{cluster_id}")
    os.makedirs(cluster_folder, exist_ok=True)

    for idx, (val, key) in enumerate(items):
        img = Image.open(val["path"])
        xmin, ymin, xmax, ymax = val["bbox"]
        crop = img.crop((
            max(0, xmin - 0.15 * (xmax - xmin)),
            max(0, ymin - 0.15 * (ymax - ymin)),
            min(img.width, xmax + 0.15 * (xmax - xmin)),
            min(img.height, ymax + 0.15 * (ymax - ymin))
        ))

        filename = os.path.join(cluster_folder, os.path.basename(val["path"]).replace(".jpg", "") + f"_{idx}.jpg")
        
        val["path_crop"] = filename
        
        crop.save(filename)

In [ ]:
im = []
for root, _, files in os.walk("clusters_kmeans_1000"):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            try:

                im.append((None, os.path.join(root, file)))
            except ValueError:
                continue
im.sort()

alm = [p for _, p in im]

print(len(alm))

*To clean up the automated results, configure the clusters_kmeans_1000 folder in revisor.js and disable group mode and annotations. Remove misclassified poses, move elements between different categories, and keep only semantically coherent clusters. These will become the building blocks of subsequent steps.*

### STEP 2: Leveraging the clusters created previously, I divide the images by activities that people are doing

A scene is not simply the sum of the poses present: a person sitting alone has a different meaning from two people sitting and conversing. Each image is represented as a vector of detected poses, capturing both the type and number of people involved.

The ordering by size of people reflects the assumption that the main actors are more visible.

In [ ]:
def get_directories(path):
    directories = []
    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        if os.path.isdir(item_path):
            directories.append(item_path)
    return directories

number_path = {path.split("/")[-1]: path for path in get_directories("clusters_kmeans_1000")}

def sort_key(path):
    try:
        return int(path.split("/")[-1].split("_")[-1])
    except ValueError:
        return float('inf')

sorted_directories = sorted(number_path.keys(), key=sort_key)
sorted_directories = [x for x in sorted_directories if "undefined" not in x]


features_indexes = {cluster_path: i for i, cluster_path in enumerate(sorted_directories)}

path_cluster = dict()

def concat_list_elements_to_string(lst):
    string = ""
    for element in lst:
        string += str(element) + "_"
    return string[:-1]  

def get_image_size(image_path):
    try:
        with Image.open(image_path) as img:
            width, height = img.size
            return width * height
    except Exception as e:
        print(f"Errore nel leggere l'immagine {image_path}: {e}")
        return 0

temp_cluster_data = dict()

for cluster_path in sorted_directories:
    for image_path in os.listdir(number_path[cluster_path]):
        if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            zero_image_path = concat_list_elements_to_string(image_path.split("/")[-1].split("_")[:-1])+"."+image_path.split("/")[-1].split(".")[1]
            full_image_path = os.path.join(number_path[cluster_path], image_path)
            image_size = get_image_size(full_image_path)
            cluster_info = (features_indexes[cluster_path], image_size, full_image_path)
            
            if zero_image_path in temp_cluster_data.keys():
                temp_cluster_data[zero_image_path].append(cluster_info)
            else:
                temp_cluster_data[zero_image_path] = [cluster_info]

for zero_image_path, cluster_infos in temp_cluster_data.items():
    sorted_infos = sorted(cluster_infos, key=lambda x: x[1], reverse=True)
    path_cluster[os.path.join(PATH_IMAGES, zero_image_path)] = [info[0] for info in sorted_infos]

            
for image_path in all_image_paths:
    if image_path not in path_cluster.keys():
        path_cluster[image_path] = []

features = []
img_path_feature = []

n_max_detections = max([len(v) for v in path_cluster.values()])
BASE_LISTS_PERSONS = [-1 for _ in range(n_max_detections) ]

for key in tqdm(path_cluster.keys(), desc="Preparing features"):
    values = path_cluster[key]

    list_ = BASE_LISTS_PERSONS.copy()
    for i, val in enumerate(values):
        if i < len(list_): list_[i] = val

    if len(values) <= 0:
        print("0 VALUES")

    features.append(list_)
    img_path_feature.append(key)

features = np.array(features)

features.shape

In [ ]:
features_std = StandardScaler().fit_transform(features)
pca = PCA(n_components=min(features.shape[0], 20, features.shape[1]), random_state=42)
features_pca = pca.fit_transform(features_std)

In [ ]:
# To calculate the perfect number of clusters - recommended anyway to consider a high number of clusters

inertias = []
K_range = range(1, 11, 1)
silhouette_scores = []

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(features_pca)
    inertias.append(km.inertia_)

    labels_tmp = km.labels_
    n_labels = len(set(labels_tmp))

    if n_labels > 1 and n_labels < len(features_pca):
        score = silhouette_score(features_pca, labels_tmp)
    else:
        score = float('nan')
    silhouette_scores.append(score)

plt.plot(K_range, inertias, 'o-')
plt.xlabel('k')
plt.ylabel('Inertia')
plt.title('Elbow method')
plt.show()

plt.plot(K_range, silhouette_scores, 'o-')
plt.xlabel('k')
plt.ylabel('Silhouette Score')
plt.title('Silhouette analysis')
plt.show()

In [ ]:
k = 500 
kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
labels_multi_persons = kmeans.fit_predict(features_pca)

centroids = kmeans.cluster_centers_
dists = cdist(centroids, centroids)

linkage_matrix = linkage(centroids, method='ward')
ordered_cluster_ids = leaves_list(linkage_matrix) 

label_map = {old: new for new, old in enumerate(ordered_cluster_ids)}
labels_multi_persons = np.array([label_map[label] for label in labels_multi_persons])

In [ ]:
output_dir = "clusters_kmeans_500_multi"

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

os.makedirs(output_dir, exist_ok=True)

for i, label in tqdm(enumerate(labels_multi_persons), desc="Creating clusters", total=len(labels_multi_persons)):
    cluster_folder = os.path.join(output_dir, f"cluster_{label}")
    os.makedirs(cluster_folder, exist_ok=True)

    shutil.copy(img_path_feature[i], os.path.join(cluster_folder, os.path.basename(img_path_feature[i])))

*Configure clusters_kmeans_500_multi keeping advanced modes disabled. Verify that each cluster represents a recurring domestic situation like watching TV, cooking, working at computer. Move images that don't match the dominant pattern of the cluster.*

### STEP 3: I divide into clusters the small groups in images of 3 by 3, furthermore I add other features

Consecutive triplets of images capture complete activity sequences: cooking is not just a person at the stove, but preparation, cooking and cleaning. Home Assistant data adds the necessary environmental context to disambiguate visually identical situations: watching TV with lights off in the evening is a different routine compared to the afternoon with natural lighting.

*The CSV file contains temporal data synchronized from the Home Assistant system: device states and room sensors. This data is correlated with images by timestamp, creating a multimodal context.*

In [ ]:
import pandas as pd

csv = pd.read_csv(PATH_CSV)

csv['timestamp'] = pd.to_datetime(csv['timestamp'])

def parse_image_timestamp(path):
    filename = path.split("/")[-1].replace(".jpg", "")
    dt = datetime.strptime(filename, "%d-%m_%H-%M-%S")
    dt = dt.replace(year=2025)
    return dt

info_dict = {}

for path in tqdm(all_image_paths):
    image_time = parse_image_timestamp(path)
    csv['delta'] = (csv['timestamp'] - image_time).abs()
    candidates = csv[csv['delta'] <= timedelta(milliseconds=750)]

    if not candidates.empty:
        closest_row = candidates.sort_values('delta').iloc[0]
        info_dict[path] = closest_row.drop('delta').to_dict()
    else:
        info_dict[path] = None

In [ ]:
from sklearn.preprocessing import LabelEncoder

keys = list(next(v for v in info_dict.values() if v is not None).keys())
encoders = {k: LabelEncoder() for k in keys if k != 'timestamp'}

for k in encoders:
    values = [info[k] for info in info_dict.values() if info is not None]
    encoders[k].fit(values)

In [ ]:
def get_cluster_dirs(path):
    return [os.path.join(path, d) for d in os.listdir(path) 
            if os.path.isdir(os.path.join(path, d)) and d.startswith("cluster_")]

def extract_cluster_num(path):
    return int(path.split("_")[-1])

def get_original_name(filename):
    return "_".join(filename.split("_")[:-1]) + "." + filename.split(".")[-1]

def get_image_size(path):
    try:
        with Image.open(path) as img:
            return img.size[0] * img.size[1]
    except:
        return 0

img_path_feature = all_image_paths.copy()

output_dir = "clusters_kmeans_500_multi"
cluster_dirs = get_cluster_dirs(output_dir)

image_clusters = {}

for cluster_dir in cluster_dirs:
    cluster_label = extract_cluster_num(cluster_dir)
    
    for filename in os.listdir(cluster_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            original_name = get_original_name(filename)
            full_path = os.path.join(cluster_dir, filename)
            size = get_image_size(full_path)
            
            cluster_info = (cluster_label, size, full_path)
            
            if original_name in image_clusters:
                image_clusters[original_name].append(cluster_info)
            else:
                image_clusters[original_name] = [cluster_info]

path_to_cluster = {}
for original_name, infos in image_clusters.items():
    best_cluster = max(infos, key=lambda x: x[1])[0]
    full_path = os.path.join(PATH_IMAGES, original_name)
    path_to_cluster[full_path] = best_cluster

labels_multi_persons = []
for img_path in img_path_feature:
    labels_multi_persons.append(path_to_cluster.get(img_path, -1))

with open("person_detections_per_image.pkl", "rb") as f:
    person_detections_per_image = pickle.load(f)

In [ ]:
from bisect import bisect_left

def extract_temporal_features(timestamp):
    hour = timestamp.hour
    weekday = timestamp.weekday()

    month = timestamp.month
    if month in [12, 1, 2]:
        season = 0  # winter
    elif month in [3, 4, 5]:
        season = 1  # spring
    elif month in [6, 7, 8]:
        season = 2  # summer
    else:
        season = 3  # autumn

    return hour, weekday, season

timestamps = [dt for dt, _ in image_info]
paths = all_image_paths

grouped_triplets = []
tolerance = timedelta(seconds=30)
for i in range(2, len(timestamps)):
    t0 = timestamps[i]
    t1, t2 = t0 - timedelta(minutes=1), t0 - timedelta(minutes=2)

    idx1 = max(0, bisect_left(timestamps, t1) - 1)
    while idx1 > 0 and abs(timestamps[idx1] - t1) > abs(timestamps[idx1 - 1] - t1):
        idx1 -= 1
    idx2 = max(0, bisect_left(timestamps, t2) - 1)
    while idx2 > 0 and abs(timestamps[idx2] - t2) > abs(timestamps[idx2 - 1] - t2):
        idx2 -= 1
    if abs(timestamps[idx1] - t1) <= tolerance and abs(timestamps[idx2] - t2) <= tolerance:
        grouped_triplets.append([paths[idx2], paths[idx1], paths[i]])

scene_histograms = []
scene_groups = []

for group in tqdm(grouped_triplets, desc="Building histograms"):
    hist = []
    for img_path in group:
        # scene_emb = extract_scene_embedding(img_path) # TO USE ALSO THE SCENE EMBEDDING
        img_idx = img_path_feature.index(img_path) if img_path in img_path_feature else -1
        person_cluster_label = labels_multi_persons[img_idx] if img_idx >= 0 else -1
        num_persons = len(person_detections_per_image.get(img_path, []))

        # combined_features = np.concatenate([
        #     scene_emb,                    # 7 dimensioni68
        #     [person_cluster_label]*20,       # 20 dimensione  
        #     [num_persons]*20                 # 20 dimensione
        # ]) -> WHEN IS USED ALSO scene_emb

        feature_list = [person_cluster_label, num_persons] * 5
        csv_info = info_dict.get(img_path)


        if csv_info is not None:
            ts = pd.to_datetime(csv_info['timestamp'])
            hour, weekday, season = extract_temporal_features(ts)
            feature_list.extend([hour, weekday])

            for k in keys:
                if k in ["timestamp", "light.lampada" ,"light.letto","light.scrivania","switch.monitor", "delta", "sensor.luminosita_camera"]:
                    continue
                v = csv_info[k]
                if k in encoders:
                    encoded_val = encoders[k].transform([v])[0]
                    feature_list.append(encoded_val)
                else:
                    feature_list.append(float(v))
        else:
            break

        combined_features = np.array(feature_list)


        hist.append(combined_features)
        

    if len(hist) != 3:
        continue
        
    scene_histograms.append(np.concatenate(hist))
    scene_groups.append(group)

In [ ]:
H = np.stack(scene_histograms)
print(H.shape)

scaler = StandardScaler()
H_scaled = scaler.fit_transform(H)

pca = PCA(n_components=20, random_state=42)
H_pca = pca.fit_transform(H_scaled)

In [ ]:
# To calculate the perfect number of clusters - recommended anyway to consider a high number of clusters

K_range_scenes = range(10, 201, 5)  # es. [10,20,...,100]
inertias_scenes = []
silhouette_scenes = []

for k in K_range_scenes:
    km = KMeans(n_clusters=k, random_state=42, n_init=10).fit(H_pca)
    inertias_scenes.append(km.inertia_)
    if k > 1 and k < len(H_pca):
        labels_s = km.labels_
        silhouette_scenes.append(silhouette_score(H_pca, labels_s))
    else:
        silhouette_scenes.append(np.nan)

# Plot Elbow 
plt.figure()
plt.plot(list(K_range_scenes), inertias_scenes, 'o-')
plt.xlabel('k (scene clusters)')
plt.ylabel('Inertia')
plt.title('Elbow Method — Level 2')
plt.show()

# Plot Silhouette
plt.figure()
plt.plot(list(K_range_scenes), silhouette_scenes, 'o-')
plt.xlabel('k (scene clusters)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Analysis — Level 2')
plt.show()

In [ ]:
k = 300 
kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
labels_scenes = kmeans.fit_predict(H_pca)

centroids = kmeans.cluster_centers_
dists = cdist(centroids, centroids)

linkage_matrix = linkage(centroids, method='ward')
ordered_cluster_ids = leaves_list(linkage_matrix) 

label_map = {old: new for new, old in enumerate(ordered_cluster_ids)}
labels_scenes = np.array([label_map[label] for label in labels_scenes])


In [ ]:
shutil.rmtree("cluster_triplet", ignore_errors=True)
os.makedirs("cluster_triplet", exist_ok=True)

for idx, (group, lbl) in tqdm(enumerate(zip(scene_groups, labels_scenes)), total=len(scene_groups)):
    out_dir = os.path.join("cluster_triplet", f"cluster_{lbl}", f"group_{idx}")
    os.makedirs(out_dir, exist_ok=True)
    for img in group:
        shutil.copy(img, os.path.join(out_dir, os.path.basename(img)))

*Activate group mode and annotations in revisor.js and configure cluster_triplet. Assign semantic labels to routines ("family breakfast", "evening work") and configure automatic actions for each scenario. Each cluster must represent a complete behavioral pattern with its environmental triggers.*